## Application of Trained DNN

Once the DNN models are trained, we implement validation and inference stages. We do this for one trained model at a time, so select the most optimal one for application.

In [ ]:
import os

import chainer
from chainer import configuration
from chainer.dataset import convert
import chainer.links as L
import chainer.functions as F
from chainer import serializers

import pickle
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt 
import random

import import_ipynb
import multi_mod_compact
from multi_mod_compact import MLP1, MLP2, MLP3
device = 0

In [ ]:
# Load the chosen architecture
dnn_final = MLP1()
dnn_final = L.Classifier(dnn_final)
dnn_final.to_device(device)

# Load the parameters of the trained DNN
directory = 'Generalization'
serializers.load_npz(os.path.join(directory,'MLP1.model'), dnn_final)
dnn_final = dnn_final.predictor

### Validation Stage

We construct the confusion matrix of the trained DNN. For this task, we use a separate validation dataset. This is generated the same way that the testing dataset was generated. Make sure to accomplish that first before this.

In [ ]:
#Load the validation datasets

dataset_dir = 'Datasets'
T00A = pickle.load(open(os.path.join(dataset_dir,'T00Ainputs_valid.pkl'),"rb"))
T00A = cp.asarray(T00A, dtype=np.float32)
T00B = pickle.load(open(os.path.join(dataset_dir,'T00Binputs_valid.pkl'),"rb"))
T00B = cp.asarray(T00B, dtype=np.float32)
P01 = pickle.load(open(os.path.join(dataset_dir,'P01inputs_valid.pkl'),"rb"))
P01 = cp.asarray(P01, dtype=np.float32)
P02 = pickle.load(open(os.path.join(dataset_dir,'P02inputs_valid.pkl'),"rb"))
P02 = cp.asarray(P02, dtype=np.float32)
P03 = pickle.load(open(os.path.join(dataset_dir,'P03inputs_valid.pkl'),"rb"))
P03 = cp.asarray(P03, dtype=np.float32)

In [ ]:
# Feed the validation datasets into the trained DNN and obtain the predicted labels
# We do this separately for each classification so that we know their proper labels

P01_labels = []
for n in range(len(P01)):
    label = dnn_final(P01)[n].array.argmax()
    P01_labels.append(label.tolist())
    
P02_labels = []
for n in range(len(P02)):
    label = dnn_final(P02)[n].array.argmax()
    P02_labels.append(label.tolist())
    
P03_labels = []
for n in range(len(P03)):
    label = dnn_final(P03)[n].array.argmax()
    P03_labels.append(label.tolist())
    
T00A_labels = []
for n in range(len(T00A)):
    label = dnn_final(T00A)[n].array.argmax()
    T00A_labels.append(label.tolist())

T00B_labels = []
for n in range(len(T00B)):
    label = dnn_final(T00B)[n].array.argmax()
    T00B_labels.append(label.tolist())

In [ ]:
# Construct the confusion matrix
# We count the number of data from each classification that was predicted as a particular label

table_data = np.array([
    [T00A_labels.count(0), T00B_labels.count(0), P01_labels.count(0), P02_labels.count(0), P03_labels.count(0)],
    [T00A_labels.count(1), T00B_labels.count(1), P01_labels.count(1), P02_labels.count(1), P03_labels.count(1)],
    [T00A_labels.count(2), T00B_labels.count(2), P01_labels.count(2), P02_labels.count(2), P03_labels.count(2)],
    [T00A_labels.count(3), T00B_labels.count(3), P01_labels.count(3), P02_labels.count(3), P03_labels.count(3)]])

table_col = ['T00A', 'T00B', 'P01', 'P02', 'P03']
table_row = [' 0 ', ' 1 ', ' 2 ', ' 3 ']

fig, ax = plt.subplots()
im = ax.imshow(table_data, cmap='viridis')

ax.set_xticks(np.arange(len(table_col)), labels=table_col, fontsize = 20)
ax.set_yticks(np.arange(len(table_row)), labels=table_row, fontsize = 20)

for i in range(len(table_col)):
    for j in range(len(table_row)):
        if (i==2 and j==1) or (i==3 and j==2) or (i==4 and j==3):
            text = ax.text(i, j, table_data[j, i], ha="center", va="center", color="k", fontsize = 18)
        else:
            text = ax.text(i, j, table_data[j, i], ha="center", va="center", color="w", fontsize = 18)

fig.text(0.5, 0.9, 'True Label', ha='center', fontsize=16)
fig.text(0.04, 0.5, 'Predicted Label', va='center', rotation='vertical', fontsize=16)

plt.show() 

### Inference Stage

We perform inference by feeding experimental data into the trained DNN and obtaining its predictions. We use this to intrepret our exotic signal.

In [ ]:
# Load the experimental data
LHCb_data = np.loadtxt('LHCb_Data.csv', skiprows=1, delimiter=',')
invmass_low = LHCb_data[:, 1]
invmass_high = LHCb_data[:, 2]
weighted_candidates = LHCb_data[:, 3]
upper_err = LHCb_data[:, 4]

# Define region of interest
energy_low = 4200.0
energy_high = 4350.0

# Randomly generate energy points from experimental data
def gen_Eaxis():
    x_data = []
    
    for i in range(len(invmass)):
        # We use a uniform distribution
        x_val = random.uniform(invmass_low[i],invmass_high[i])
        x_data.append(x_val)
    
    return np.array(x_data)

# Randomly generate intensity points from experimental data
def gen_Amp():
    y_data = []

    for i in range(len(weighted_candidates)):
        # We use a normal distribution
        y_val = np.random.normal(weighted_candidates[i],upper_err[i])
        y_data.append(y_val)

    return np.array(y_data)

# Combine the two and cut to region of interest
def gen_Exp():
    x_data = gen_Eaxis()
    y_data = gen_Amp()

    y_data = y_data[x_data < energy_high]
    x_data = x_data[x_data < energy_high]
    y_data = y_data[x_data >= energy_low]
    x_data = x_data[x_data >= energy_low]

    return np.concatenate((x_data,y_data), axis=0)

In [ ]:
# We construct N number of line shapes from the experimental data

N = 3000
Exp_data = []

for i in range(N):
    Exp = gen_Exp()
    Exp_data.append(Exp)

In [ ]:
# We now feed the constructed line shapes into the trained DNN and obtain predictions

Exp_data = cp.asarray(Exp_data, dtype=np.float32)

Exp_labels = []
for n in range(len(Exp_data)):
    label = dnn_final(Exp_data)[n].array.argmax()
    Exp_labels.append(label.tolist())

In [ ]:
# Display the number of obtained predictions per label

table_data = [
    [Exp_labels.count(0)], [Exp_labels.count(1)],
    [Exp_labels.count(2)], [Exp_labels.count(3)], [3000]]

table_col = ['$P_\psi^N(4312)^+$']
table_row = [' 0 ', ' 1 ', ' 2 ', ' 3 ', ' Total ']

fig, ax = plt.subplots(figsize=(3, 2))
ax.set_axis_off() 
table = ax.table(cellText = table_data, rowLabels = table_row, colLabels = table_col, rowLoc ='right', loc ='upper left',
                rowColours = ['lightsteelblue']*4+['silver'], colColours = ['lightsteelblue'])

table.scale(1,1.5)

plt.show() 